<div align="center">

# Problem Set 02

</div>

---

# PARTE I
## Procesamiento de Lenguaje Natural (NLP) y *Embeddings*

---

In [49]:
import os
# Forzamos a TensorFlow a usar el motor antiguo de Keras 2, ya que no esta disponible para el kernel de python 3.14.2
os.environ["TF_USE_LEGACY_KERAS"] = "1"

import tensorflow as tf
import tf_keras as keras  # Usamos el paquete de compatibilidad
import transformers

print(f"Versión de TF: {tf.__version__}")
# Esto me debería mostrar algo como 2.16.x o 2.17.x pero usando el motor de tf-keras
# Despues de correr esto podemos ejecutar el siguiente fragmento de codigo porque sin esto me seguira dando el error

Versión de TF: 2.16.2


In [66]:

import pandas as pd
import numpy as np
import re
from gensim.models import Word2Vec
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import f1_score, classification_report
from sentence_transformers import SentenceTransformer

# ==========================================
# 1. Carga y Preprocesamiento de Datos
# ==========================================
print("=" * 50)
print("=" * 6 + " 1. Carga y Preprocesamiento de Datos " + "=" * 6)
print("=" * 50)

df = pd.read_csv('retail_reviews.csv')

def clean_text(text):
    if pd.isna(text): return ""
    # Eliminar caracteres especiales tipo #@#&* y números
    text = re.sub(r'[^a-zA-ZáéíóúñÁÉÍÓÚÑ ]', '', text)
    # Convertir a minúsculas y quitar espacios extra
    return text.lower().strip()

df['text_clean'] = df['text'].apply(clean_text)
# Eliminar filas vacías tras la limpieza
df = df[df['text_clean'] != ""]

# ==========================================
# 1.1 Análisis de Embeddings (Word2Vec)
# ==========================================
print("=" * 51)
print("=" * 6 + " 1.1 Análisis de Embeddings (Word2Vec) " + "=" * 6)
print("=" * 51)

# Tokenización para Word2Vec
tokenized_corpus = [doc.split() for doc in df['text_clean']]

# Entrenamiento del modelo Word2Vec
# vector_size: dimensión del vector, window: contexto, min_count: frecuencia mínima
w2v_model = Word2Vec(sentences=tokenized_corpus, vector_size=100, window=5, min_count=1, workers=4)

print("--- 1.1.a) Términos Similares ---")
palabras_clave = ["defectuoso", "rápido"]
for palabra in palabras_clave:
    if palabra in w2v_model.wv:
        similares = w2v_model.wv.most_similar(palabra, topn=5)
        print(f"\nSimilares a '{palabra}':")
        for p, sim in similares:
            print(f" - {p}: {sim:.4f}")

print("\n--- 1.1.c) Álgebra Vectorial (Analogía Retail) ---")
# Analogía: "excelente" - "positivo" + "negativo" debería tender a algo como "malo" o "pésimo"
try:
    resultado_algebra = w2v_model.wv.most_similar(positive=['excelente', 'negativo'], negative=['positivo'], topn=1)
    print(f"Operación: 'excelente' - 'positivo' + 'negativo'")
    print(f"Resultado semántico: {resultado_algebra[0][0]} (Similitud: {resultado_algebra[0][1]:.4f})")
except KeyError as e:
    print(f"Error en analogía: {e}")

# ==========================================
# 1.2 Clasificación de Texto
# ==========================================
print("=" * 40)
print("=" * 6 + " 1.2 Clasificación de Texto " + "=" * 6)
print("=" * 40)

# Preparación de etiquetas (Label Encoding manual para Positivo/Negativo)
df['label'] = df['sentiment'].map({'Positivo': 1, 'Negativo': 0})
df = df.dropna(subset=['label']) # Limpiar si hay etiquetas mal formadas

X_train, X_test, y_train, y_test = train_test_split(
    df['text_clean'], df['label'], test_size=0.2, random_state=42
)

# --- Enfoque A: Baseline TF-IDF + Regresión Logística ---
tfidf = TfidfVectorizer(max_features=1000)
X_train_tfidf = tfidf.fit_transform(X_train)
X_test_tfidf = tfidf.transform(X_test)

model_lr = LogisticRegression()
model_lr.fit(X_train_tfidf, y_train)
y_pred_tfidf = model_lr.predict(X_test_tfidf)
f1_tfidf = f1_score(y_test, y_pred_tfidf)

# --- Enfoque B: BERT Embeddings + Regresión Logística ---
# Usamos un modelo ligero de BERT (paraphrase-multilingual-MiniLM-L12-v2)
print("\nGenerando embeddings de BERT (esto puede tardar un poco)...")
bert_model = SentenceTransformer('paraphrase-multilingual-MiniLM-L12-v2')

X_train_bert = bert_model.encode(X_train.tolist())
X_test_bert = bert_model.encode(X_test.tolist())

model_bert = LogisticRegression()
model_bert.fit(X_train_bert, y_train)
y_pred_bert = model_bert.predict(X_test_bert)
f1_bert = f1_score(y_test, y_pred_bert)

# ==========================================
# Reporte Final del Item 1
# ==========================================
print("=" * 40)
print("=" * 6 + " Reporte Final del Item 1 " + "=" * 6)
print("=" * 40)

print("\n" + "="*30)
print("REPORTE DE COMPARACIÓN")
print("="*30)
print(f"F1-Score TF-IDF (Baseline): {f1_tfidf:.4f}")
print(f"F1-Score BERT Embeddings:    {f1_bert:.4f}")
print("-" * 30)

# Identificar un caso donde TF-IDF falla y BERT acierta
print("\n--- Análisis de Fallos Específicos ---")
for i in range(len(y_test)):
    idx = y_test.index[i]
    if y_pred_tfidf[i] != y_test.iloc[i] and y_pred_bert[i] == y_test.iloc[i]:
        print(f"Texto: '{df.loc[idx, 'text']}'")
        print(f"Real: {y_test.iloc[i]} | TF-IDF predijo: {y_pred_tfidf[i]} | BERT predijo: {y_pred_bert[i]}")
        break

====== 1. Carga y Preprocesamiento de Datos ======
====== 1.1 Análisis de Embeddings (Word2Vec) ======
--- 1.1.a) Términos Similares ---

Similares a 'defectuoso':
 - insatisfecho: 0.9882
 - una: 0.9869
 - total: 0.9857
 - sucio: 0.9846
 - decepción: 0.9817

Similares a 'rápido':
 - recomendado: 0.9851
 - llegó: 0.9828
 - estado: 0.9819
 - eficaz: 0.9811
 - y: 0.9784

--- 1.1.c) Álgebra Vectorial (Analogía Retail) ---
Error en analogía: "Key 'negativo' not present in vocabulary"
====== 1.2 Clasificación de Texto ======

Generando embeddings de BERT (esto puede tardar un poco)...
====== Reporte Final del Item 1 ======

REPORTE DE COMPARACIÓN
F1-Score TF-IDF (Baseline): 0.9000
F1-Score BERT Embeddings:    0.8924
------------------------------

--- Análisis de Fallos Específicos ---
Texto: '  Producto duradero pero el precio es elevado.  '
Real: 0.0 | TF-IDF predijo: 1.0 | BERT predijo: 0.0


---

# PARTE II
## Topic Modeling (No supervisado)

---

In [ ]:
# 1. Instalación de librerías
# Para poder ejecutar este fragmento de codigo se tiene que instalar la siguiente libreria 
# !pip install bertopic sentence-transformers pandas

import pandas as pd
from bertopic import BERTopic
from sentence_transformers import SentenceTransformer
from umap import UMAP
from hdbscan import HDBSCAN
import re

# ==========================================
# 1. Carga y Limpieza (Igual que Parte 1)
# ==========================================
print("=" * 54)
print("=" * 6 + " 2.1 Carga y Limpieza (Igual que Parte 1) " + "=" * 6)
print("=" * 54)

df = pd.read_csv('retail_reviews.csv')

# Limpieza básica para quitar ruido visual
def clean_text(text):
    if pd.isna(text): return ""
    text = re.sub(r'[^a-zA-ZáéíóúñÁÉÍÓÚÑ ]', '', text)
    return text.lower().strip()

df['text_clean'] = df['text'].apply(clean_text)
docs = df[df['text_clean'] != ""]['text_clean'].tolist()

# ==========================================
# 2. Configuración del Pipeline BERTopic
# ==========================================
print("=" * 53)
print("=" * 6 + " 2.2 Configuración del Pipeline BERTopic " + "=" * 6)
print("=" * 53)

# Paso 1: Embeddings (Multilingüe para Español)
# Usamos un modelo que soporte español para captar semántica correcta
embedding_model = SentenceTransformer("paraphrase-multilingual-MiniLM-L12-v2")
embeddings = embedding_model.encode(docs, show_progress_bar=True)

# Paso 2: UMAP (Reducción de Dimensionalidad)
# n_neighbors=15: Balance entre estructura local y global
# n_components=5: Reducimos a 5 dimensiones para que HDBSCAN trabaje bien
umap_model = UMAP(n_neighbors=15, n_components=5, min_dist=0.0, metric='cosine', random_state=42)

# Paso 3: HDBSCAN (Clusterización)
# min_cluster_size=10: Queremos tópicos con al menos 10 reseñas
# prediction_data=True: Para poder predecir nuevos documentos luego
hdbscan_model = HDBSCAN(min_cluster_size=10, metric='euclidean', cluster_selection_method='eom', prediction_data=True)

# ==========================================
# 3. Entrenamiento del Modelo
# ==========================================
print("=" * 42)
print("=" * 6 + " 2.3 Entrenamiento del Modelo " + "=" * 6)
print("=" * 42)
topic_model = BERTopic(
    embedding_model=embedding_model, # Paso 1
    umap_model=umap_model,           # Paso 2
    hdbscan_model=hdbscan_model,     # Paso 3
    language="multilingual",         # Refuerzo para stopwords en español
    calculate_probabilities=True,
    verbose=True
)

topics, probs = topic_model.fit_transform(docs, embeddings)

# ==========================================
# 4. Resultados e Interpretación de Negocio
# ==========================================
print("=" * 56)
print("=" * 6 + " 2.4 Resultados e Interpretación de Negocio " + "=" * 6)
print("=" * 56)

# Generaramos la tabla de Top 5 Tópicos
freq = topic_model.get_topic_info()
print("\n--- Top 5 Tópicos Encontrados ---")
print(freq.head(6)) # head(6) porque el primero suele ser el -1 (Ruido)

# Mostrar palabras clave del Tópico 0 (el más frecuente)
print("\n--- Palabras Clave del Tópico Principal (ID 0) ---")
print(topic_model.get_topic(0))

# Visualización pora que nos funcione en Jupyter/Colab
topic_model.visualize_barchart(top_n_topics=5)
topic_model.visualize_topics()

====== 2.1 Carga y Limpieza (Igual que Parte 1) ======
====== 2.2 Configuración del Pipeline BERTopic ======


Batches:   0%|          | 0/75 [00:00<?, ?it/s]

2026-01-30 09:48:16,998 - BERTopic - Dimensionality - Fitting the dimensionality reduction algorithm


====== 2.3 Entrenamiento del Modelo ======


2026-01-30 09:48:38,891 - BERTopic - Dimensionality - Completed ✓
2026-01-30 09:48:38,898 - BERTopic - Cluster - Start clustering the reduced embeddings
2026-01-30 09:48:39,189 - BERTopic - Cluster - Completed ✓
2026-01-30 09:48:39,197 - BERTopic - Representation - Fine-tuning topics using representation models.
2026-01-30 09:48:39,240 - BERTopic - Representation - Completed ✓


====== 2.4 Resultados e Interpretación de Negocio ======

--- Top 5 Tópicos Encontrados ---
   Topic  Count                                   Name  \
0      0    132         0_contento_empacado_bien_buena   
1      1    121           1_son_perfectos_compra_color   
2      2    113          2_estado_en_recomendado_llegó   
3      3    112     3_eficaz_empacado_bien_recomendado   
4      4    111  4_funciona_excelentemente_duradero_es   
5      5     96               5_fue_envío_buena_rápido   

                                      Representation  \
0  [contento, empacado, bien, buena, rápido, de, ...   
1  [son, perfectos, compra, color, tamaño, excele...   
2  [estado, en, recomendado, llegó, rápido, excel...   
3  [eficaz, empacado, bien, recomendado, llegó, p...   
4  [funciona, excelentemente, duradero, es, el, p...   
5  [fue, envío, buena, rápido, el, es, de, muy, c...   

                                 Representative_Docs  
0  [rápido bien empacado y de buena calidad muy c... 

---

# PARTE III
## Sistemas de Recomendación: Filtrado Colaborativo Explícito

---

In [62]:
import pandas as pd
from surprise import SVD, Dataset, Reader, accuracy
from surprise.model_selection import cross_validate, train_test_split


# ==========================================
# 3.1 Factorización Matricial (SVD)
# ==========================================
print("=" * 43)
print("=" * 4 + " 3.1 Factorización Matricial (SVD) " + "=" * 4)
print("=" * 43)

# 1. Carga y Limpieza (Eliminar outliers como el 999 detectado mas arriba)
df_ratings = pd.read_csv('video_ratings.csv')
df_ratings = df_ratings[df_ratings['rating'] <= 5] # Filtramos solo valores 1-5

# 2. Configurar Surprise
reader = Reader(rating_scale=(1, 5))
data = Dataset.load_from_df(df_ratings[['user_id', 'movie_id', 'rating']], reader)

# 3. Implementación de SVD y Validación Cruzada (5-fold)
algo = SVD()
results = cross_validate(algo, data, measures=['RMSE', 'MAE'], cv=5, verbose=True)

rmse_promedio = results['test_rmse'].mean()
print(f"\n> RMSE Promedio tras 5-fold: {rmse_promedio:.4f}")


# ==========================================
# 3.2 El problema del Cold-Start
# ==========================================
print("=" * 44)
print("=" * 6 + " 3.2 El problema del Cold-Start " + "=" * 6)
print("=" * 44)

# Dividir en entrenamiento y prueba (80/20)
trainset, testset = train_test_split(data, test_size=0.2, random_state=42)
algo.fit(trainset)
predictions = algo.test(testset)

# Identificar usuarios fríos (< 3 interacciones en el set original)
user_counts = df_ratings['user_id'].value_counts()
cold_users = user_counts[user_counts < 3].index

# Filtrar predicciones de test que corresponden a usuarios fríos
cold_predictions = [p for p in predictions if p.uid in cold_users]

# Calcular RMSE para este subgrupo
rmse_cold = accuracy.rmse(cold_predictions)
print(f"RMSE para usuarios Cold-Start: {rmse_cold:.4f}")

==== 3.1 Factorización Matricial (SVD) ====
Evaluating RMSE, MAE of algorithm SVD on 5 split(s).

                  Fold 1  Fold 2  Fold 3  Fold 4  Fold 5  Mean    Std     
RMSE (testset)    0.6901  0.7290  0.6838  0.6826  0.7052  0.6982  0.0174  
MAE (testset)     0.3815  0.3809  0.3605  0.3661  0.3805  0.3739  0.0088  
Fit time          0.20    0.18    0.17    0.17    0.35    0.21    0.07    
Test time         0.02    0.02    0.02    0.02    0.02    0.02    0.00    

> RMSE Promedio tras 5-fold: 0.6982
====== 3.2 El problema del Cold-Start ======
RMSE: 0.1747
RMSE para usuarios Cold-Start: 0.1747


---

# PARTE IV
## Sistemas de Recomendación: Feedback Implícito y LTR

---

In [61]:
import pandas as pd
import scipy.sparse as sparse
from implicit.als import AlternatingLeastSquares
import os

# ==========================================
# 4.1 Mínimos Cuadrados Alternados (ALS)
# ==========================================
print("=" * 52)
print("=" * 6 + " 4.1 Mínimos Cuadrados Alternados (ALS) " + "=" * 6)
print("=" * 52)

# Paso opcional: Evita problemas de hilos en Anaconda/Mac
os.environ['MKL_NUM_THREADS'] = '1'

# 1. Limpieza y conversión de tipos
df_music = pd.read_csv('music_logs.csv').dropna()
df_music = df_music[df_music['play_count'] > 0].copy()

# 2. Crear la matriz Ítem-Usuario directamente
# Convertimos play_count a float32 para silenciar el Warning que nos aparecia
user_items = sparse.csr_matrix((
    df_music['play_count'].astype('float32'), 
    (df_music['song_id'].astype(int), df_music['user_id'].astype(int))
))

# 3. Entrenar el modelo
model = AlternatingLeastSquares(factors=64, regularization=0.1, iterations=20)
model.fit(user_items) 

# ==========================================
# 4.2 Evaluación de Ranking (NDCG)
# ==========================================
print("=" * 46)
print("=" * 6 + " 4.2 Evaluación de Ranking (NDCG) " + "=" * 6)
print("=" * 46)

def calculate_ndcg_at_k(relevance_scores, k):
    """
    Calcula el NDCG a un nivel K.
    relevance_scores: Lista de 1s (relevante) y 0s (no relevante) en el orden recomendado.
    """
    # 1. Calcular DCG@K
    # Formula: sum(rel_i / log2(i + 1))
    dcg = sum([rel / np.log2(idx + 2) for idx, rel in enumerate(relevance_scores[:k])])
    
    # 2. Calcular IDCG@K (El caso ideal: todos los aciertos al principio)
    ideal_relevance = sorted(relevance_scores, reverse=True)
    idcg = sum([rel / np.log2(idx + 2) for idx, rel in enumerate(ideal_relevance[:k])])
    
    # 3. Calcular NDCG
    return dcg / idcg if idcg > 0 else 0


# Suponiendo que el modelo nos recomendó 5 canciones y así se ven en el 'test set':
# Canción 1: Escuchada (1), Canción 2: No (0), Canción 3: Escuchada (1), ...
real_user_relevance = [1, 0, 1, 1, 0] 

ndcg_5 = calculate_ndcg_at_k(real_user_relevance, k=5)

print(f"Resultados de Evaluación de Ranking:")
print(f"Relevancia real en el Top-5: {real_user_relevance}")
print(f"NDCG@5 obtenido: {ndcg_5:.4f}")


====== 4.1 Mínimos Cuadrados Alternados (ALS) ======


  0%|          | 0/20 [00:00<?, ?it/s]

====== 4.2 Evaluación de Ranking (NDCG) ======
Resultados de Evaluación de Ranking:
Relevancia real en el Top-5: [1, 0, 1, 1, 0]
NDCG@5 obtenido: 0.9060


---

# PARTE V
## Desafíos avanzados

---

In [59]:
import pandas as pd
import numpy as np
from surprise import SVD, Dataset, Reader

# ==============================================================================
# 5.1. NLP: Análisis de Sentimiento Basado en Aspectos (ABSA)
# ==============================================================================
print("=" * 81)
print("=" * 10 + " 5.1. NLP: Análisis de Sentimiento Basado en Aspectos (ABSA) " + "=" * 10)
print("=" * 81)

# Vocabularios de polaridad (Lógica de palabras exactas)
palabras_positivas = ['excelente', 'bueno', 'buena', 'gran', 'calidad', 'recomiendo', 'perfecto', 'rapido', 'satisfecho']
palabras_negativas = ['malo', 'mala', 'defectuoso', 'tardo', 'lento', 'roto', 'pobre', 'problema', 'decepcion', 'pesimo']

# Función para detectar señales mixtas usando split() para evitar falsos positivos
def es_mixto(texto):
    if pd.isna(texto): return False
    tokens = set(texto.lower().split())
    tiene_pos = any(pos in tokens for pos in palabras_positivas)
    tiene_neg = any(neg in tokens for neg in palabras_negativas)
    return tiene_pos and tiene_neg

# Filtrar y mostrar resultados
df['es_mixto'] = df['text_clean'].apply(es_mixto)
df_mixto = df[df['es_mixto']].copy()

print(f"Reseñas mixtas detectadas: {len(df_mixto)}")
print(df_mixto[['text', 'sentiment']].head(5))

# ==============================================================================
# 5.2. RecSys: Sesgo de Popularidad y Long Tail
# ==============================================================================
print("=" * 67)
print("=" * 10 + " 5.2. RecSys: Sesgo de Popularidad y Long Tail " + "=" * 10)
print("=" * 67)

# Entrenamos SVD rápido con los datos de la Parte 3
reader = Reader(rating_scale=(1, 5))
data_svd = Dataset.load_from_df(df_ratings[['user_id', 'movie_id', 'rating']], reader)
trainset = data_svd.build_full_trainset()
algo_svd = SVD(random_state=42)
algo_svd.fit(trainset)

# Calculamos cobertura para una muestra de usuarios
usuarios_test = df_ratings['user_id'].unique()[:100]
items_totales = set(df_ratings['movie_id'].unique())
items_recomendados = set()

for u_id in usuarios_test:
    # Predecir para todos los items y tomar Top 10
    preds = [(i_id, algo_svd.predict(u_id, i_id).est) for i_id in items_totales]
    preds.sort(key=lambda x: x[1], reverse=True)
    top_10 = [x[0] for x in preds[:10]]
    items_recomendados.update(top_10)

cobertura = (len(items_recomendados) / len(items_totales)) * 100
print(f"Cobertura de Catálogo: {cobertura:.2f}%")

# ==============================================================================
# 5.3. RecSys: Re-ranking Híbrido por Margen Financiero
# ==============================================================================
print("=" * 75)
print("=" * 10 + " 5.3. RecSys: Re-ranking Híbrido por Margen Financiero " + "=" * 10)
print("=" * 75)

# 1. Limpieza de Metadata
df_meta = pd.read_csv('movie_metadata.csv')
df_meta['margin_category'] = df_meta['margin_category'].fillna('Low').str.strip().str.title()
# Corregir el caso específico 'High' vs 'High' que detectamos
df_meta['margin_category'] = df_meta['margin_category'].replace({'High': 'High', 'High': 'High', 'High': 'High'})

# 2. Función de Re-ranking
def aplicar_reranking(user_id, n=10):
    # Obtener predicciones originales
    preds = [(i_id, algo_svd.predict(user_id, i_id).est) for i_id in items_totales]
    preds.sort(key=lambda x: x[1], reverse=True)
    top_original = preds[:n]
    
    # Aplicar pesos de margen
    lista_final = []
    for m_id, score in top_original:
        categoria = df_meta[df_meta['movie_id'] == m_id]['margin_category'].values[0] if m_id in df_meta['movie_id'].values else 'Low'
        
        factor = 1.2 if categoria == 'High' else 0.9
        score_final = score * factor
        lista_final.append({'movie_id': m_id, 'margin': categoria, 'score_orig': score, 'score_final': score_final})
    
    return pd.DataFrame(lista_final).sort_values('score_final', ascending=False)

# Ejecutar para el usuario 260
df_resultado = aplicar_reranking(user_id=260)
print(df_resultado)

========== 5.1. NLP: Análisis de Sentimiento Basado en Aspectos (ABSA) ==========
Reseñas mixtas detectadas: 226
                                                 text sentiment
0   Atención al cliente pésima. Producto de mala c...  Negativo
2     Atención al cliente pésima. Producto de mala...  Negativo
6   Producto caro y muy lento en funcionar. Gran d...  Negativo
11  Producto caro y muy lento en funcionar. Gran d...  Negativo
23  Atención al cliente pésima. Producto de mala c...  Negativo
========== 5.2. RecSys: Sesgo de Popularidad y Long Tail ==========
Cobertura de Catálogo: 77.69%
========== 5.3. RecSys: Re-ranking Híbrido por Margen Financiero ==========
   movie_id margin  score_orig  score_final
5       125   High    3.318060     3.981672
0        28    Low    3.385831     3.047248
1       234    Low    3.359567     3.023611
2        31    Low    3.336855     3.003169
3       245    Low    3.333733     3.000360
4         1    Low    3.324766     2.992290
6        96    Low   